# Scenario 2
A retail company receives daily sales transaction files from multiple store locations in an Azure Data Lake folder. Instead of reprocessing all historical data every day, the DE team uses **Spark Structured Streaming** to incrementally load only the newly arrived files into a Delta table. This ensures timely updates to analytics dashboards while optimizing compute costs and processing time.

## Streaming Query

In [0]:
my_schema = """
    order_id INT,
    customer_id INT,
    order_date DATE,
    amount DOUBLE
"""

In [0]:
df_batch = spark.read.format("csv")\
  .option("header", "true")\
  .schema(my_schema)\
  .load("/Volumes/pyspark_cata/source/db_volume/streamSource/")

display(df_batch)

order_id,customer_id,order_date,amount
6,100,2025-08-07,248.69
7,102,2025-08-08,243.85
8,101,2025-08-09,308.31
9,105,2025-08-10,367.45
10,105,2025-08-11,328.2
1,101,2025-08-02,246.84
2,104,2025-08-03,111.3
3,103,2025-08-04,52.0
4,103,2025-08-05,98.7
5,102,2025-08-06,392.67


In [0]:
df = spark.readStream.format("csv")\
  .option("header", "true")\
  .schema(my_schema)\
  .load("/Volumes/pyspark_cata/source/db_volume/streamSource/")

## **Streaming Output**

In [0]:
df.writeStream.format("delta")\
  .option("checkpointlocation", "/Volumes/pyspark_cata/source/db_volume/streamSink/checkpoint")\
  .option("mergeSchema", True)\
  .trigger(once=True)\
  .start("/Volumes/pyspark_cata/source/db_volume/streamSink/data")

In [0]:
%sql
SELECT * FROM delta.`/Volumes/pyspark_cata/source/db_volume/streamSink/data/`

order_id,customer_id,order_date,amount
6,100,2025-08-07,248.69
7,102,2025-08-08,243.85
8,101,2025-08-09,308.31
9,105,2025-08-10,367.45
10,105,2025-08-11,328.2
1,101,2025-08-02,246.84
2,104,2025-08-03,111.3
3,103,2025-08-04,52.0
4,103,2025-08-05,98.7
5,102,2025-08-06,392.67
